In [1]:
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
df = pd.read_csv("C:\\progetti\\low-code-ard-proj\\resyduo_components\\transformation_component\\transformation_storage\\tag_data\\surprise_tag_comp_cut_5_5.csv")

In [3]:
surprise_df = df

In [5]:
df.head()

,tagID,componentID,rating
0,solar tracker,DFRobot LCD Keypad Shield,0
1,solar tracker,GY21 Temperature and Humidity,0
2,solar tracker,Panel Rotary Potentiometer 6mm shaft diameter ...,0
3,solar tracker,USB-A to Micro-USB Cable,0
4,solar tracker,2.54_legs_female_2P,0


In [5]:
df = df[df.rating != 0] 

In [6]:
df.sort_values(by=['rating'],ascending=False).head(100)

,tagID,componentID,rating
1450701,home automation,Arduino UNO,516
2417961,robotics,Arduino UNO,362
57057,internet of things,Arduino UNO,310
1847475,arduino,Arduino UNO,271
1448956,home automation,Jumper wires (generic),251
...,...,...,...
1449263,home automation,Buzzer,54
1246854,audio,Arduino Nano R3,53
722462,1sheeld,1Sheeld,53
3274677,entertainment system,Arduino UNO,53


In [8]:
from sklearn import preprocessing
import pandas as pd
def scale_normalizer(df,min, max, column):
    scaled_df=df
    display("max",df["rating"].max())
    scaler = preprocessing.MinMaxScaler(feature_range=(min, max))
    scaled_df["rating"] = scaler.fit_transform(df[[column]])
    #scaled_df = pd.DataFrame(d, "rating")
    #display(scaled_df.head())
    #display(df.head())
    display("norm MAx ",df["rating"].max())
    return scaled_df



df = scale_normalizer(df, 1,10, "rating")

'max'

10.0

C:\Users\Skynetth\AppData\Local\Temp\ipykernel_22948\3527851949.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scaled_df["rating"] = scaler.fit_transform(df[[column]])


'norm MAx '

10.0

In [9]:
norm_surprise_df = df

In [10]:
df.sort_values(by=['rating'],ascending=False).head(100)

,tagID,componentID,rating
1450701,home automation,Arduino UNO,10.000000
2417961,robotics,Arduino UNO,7.308738
57057,internet of things,Arduino UNO,6.400000
1847475,arduino,Arduino UNO,5.718447
1448956,home automation,Jumper wires (generic),5.368932
...,...,...,...
1449263,home automation,Buzzer,1.926214
1246854,audio,Arduino Nano R3,1.908738
722462,1sheeld,1Sheeld,1.908738
3274677,entertainment system,Arduino UNO,1.908738


In [12]:
df.head()

,tagID,componentID,rating
110,solar tracker,Resistor 330 ohm,1.017476
162,solar tracker,Resistor 220 ohm,1.000000
248,solar tracker,Resistor 2.21k ohm,1.000000
337,solar tracker,HC-05 Bluetooth Module,1.000000
587,solar tracker,Through Hole Resistor 10 ohm,1.000000


In [13]:
df.shape

(43917, 3)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43917 entries, 110 to 3367455
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   tagID        43917 non-null  object 
 1   componentID  43917 non-null  object 
 2   rating       43917 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.3+ MB


In [15]:
print('Dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::200000, :])

Dataset shape: (43917, 3)
-Dataset examples-
             tagID       componentID    rating
110  solar tracker  Resistor 330 ohm  1.017476


## EDA

### Ratings Distribution

In [16]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = df['rating'].value_counts().sort_index(ascending=False)
trace = go.Bar(x = data.index,
               text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
               textposition = 'auto',
               textfont = dict(color = '#000000'),
               y = data.values,
               )
# Create layout
layout = dict(title = 'Distribution Of {} book-ratings'.format(df.shape[0]),
              xaxis = dict(title = 'Rating'),
              yaxis = dict(title = 'Count'))
# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

We can see that over 62% of all ratings in the data are 0, and very few ratings are 1 or 2, or 3, low rating books mean they are generally really bad.

### Ratings Distribution By Book

In [17]:
df.columns

Index(['tagID', 'componentID', 'rating'], dtype='object')

In [18]:
# Number of ratings per book
data = df.groupby('componentID')['rating'].count().clip(upper=50)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per Components (Clipped at 50)',
                   xaxis = dict(title = 'Number of Ratings Per Components'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [19]:
df.groupby('componentID')['rating'].count().reset_index().sort_values('rating', ascending=False)[:10]

,componentID,rating
404,Arduino UNO,1246
896,Jumper wires (generic),882
389,Arduino Nano R3,832
484,Breadboard (generic),735
1340,Resistor 10k ohm,550
934,LED (generic),531
375,Arduino Mega 2560,487
495,Buzzer,435
1810,Ultrasonic Sensor - HC-SR04 (Generic),403
1343,Resistor 1k ohm,388


Most of the books received less than 5 ratings, and very few books have many ratings, although the most rated book has received 2,502 ratings.

### Ratings Distribution By User

In [20]:
# Number of ratings per user
data = df.groupby('componentID')['rating'].count().clip(upper=50)

# Create trace
trace = go.Histogram(x = data.values,
                     name = 'Ratings',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 2))
# Create layout
layout = go.Layout(title = 'Distribution Of Number of Ratings Per User (Clipped at 50)',
                   xaxis = dict(title = 'Componente Per Tag'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

In [21]:
df.groupby('componentID')['rating'].count().reset_index().sort_values('rating', ascending=False)[:10]

,componentID,rating
404,Arduino UNO,1246
896,Jumper wires (generic),882
389,Arduino Nano R3,832
484,Breadboard (generic),735
1340,Resistor 10k ohm,550
934,LED (generic),531
375,Arduino Mega 2560,487
495,Buzzer,435
1810,Ultrasonic Sensor - HC-SR04 (Generic),403
1343,Resistor 1k ohm,388


Most of the users gave less than 5 ratings, and very few users gave many ratings, although the most productive user have given 13,602 ratings.

I'm sure you have noticed that the above two charts share the same distribution. The number of ratings per movie and the number of ratings per user decay exponentially.

To reduce the dimensionality of the dataset, we will filter out rarely rated movies and rarely rating users.

In [22]:
df.groupby('componentID')['rating'].count().reset_index().sort_values('rating', ascending=True)[:10]

,componentID,rating
356,Arduino MKR Connector Carrier,2
669,Df player mini,2
223,Adafruit 1.44 TFT,2
370,Arduino MKR WAN 1310,2
1273,QMC5883l Compass (GY-271),2
1716,Temperature sensor,2
1185,Panasonic 2 rechargeable Panasonic NCR18650B L...,2
1020,MLX90615 Digital Infrared Temperature Sensor M...,3
23,10 ohm 2W,3
22,10 White LED,3


In [23]:
min_book_ratings = 1
filter_books = df['componentID'].value_counts() > min_book_ratings
filter_books = filter_books[filter_books].index.tolist()

min_user_ratings = 1
filter_users = df['tagID'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['componentID'].isin(filter_books)) & (df['tagID'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(43917, 3)
The new data frame shape:	(43917, 3)


## Surprise

To load a dataset from a pandas dataframe, we will use the load_from_df() method, we will also need a Reader object, and the rating_scale parameter must be specified. The dataframe must have three columns, corresponding to the user ids, the item ids, and the ratings in this order. Each row thus corresponds to a given rating.

In [29]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df_new[['tagID', 'componentID', 'rating']], reader)

With the Surprise library, we will benchmark the following algorithms

### Basic algorithms

#### NormalPredictor

* NormalPredictor algorithm predicts a random rating based on the distribution of the training set, which is assumed to be normal. This is one of the most basic algorithms that do not do much work.

#### BaselineOnly

* BasiclineOnly algorithm predicts the baseline estimate for given user and item.

### k-NN algorithms

#### KNNBasic

* KNNBasic is a basic collaborative filtering algorithm.

#### KNNWithMeans

* KNNWithMeans is basic collaborative filtering algorithm, taking into account the mean ratings of each user.

#### KNNWithZScore

* KNNWithZScore is a basic collaborative filtering algorithm, taking into account the z-score normalization of each user.

#### KNNBaseline

* KNNBaseline is a basic collaborative filtering algorithm taking into account a baseline rating.

### Matrix Factorization-based algorithms

#### SVD

* SVD algorithm is equivalent to Probabilistic Matrix Factorization (http://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf)

#### SVDpp

* The SVDpp algorithm is an extension of SVD that takes into account implicit ratings.

#### NMF

* NMF is a collaborative filtering algorithm based on Non-negative Matrix Factorization. It is very similar with SVD.

### Slope One

* Slope One is a straightforward implementation of the SlopeOne algorithm. (https://arxiv.org/abs/cs/0702144)

### Co-clustering

* Co-clustering is a collaborative filtering algorithm based on co-clustering (http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.113.6458&rep=rep1&type=pdf)


We use rmse as our accuracy metric for the predictions.

In [ ]:
"""benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)"""

In [ ]:
#surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [ ]:
#surprise_results

BaselineOnly algorithm gave us the best rmse, therefore, we will proceed further with BaselineOnly and use Alternating Least Squares (ALS).

In [25]:
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
#algo = BaselineOnly(bsl_options=bsl_options)
algo = NMF()
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Using ALS


{'test_rmse': array([0.14447854, 0.09190804, 0.09234493]),
 'fit_time': (0.6800003051757812, 0.6860003471374512, 0.6395790576934814),
 'test_time': (0.07699942588806152, 0.07500290870666504, 0.15199947357177734)}

We use the train_test_split() to sample a trainset and a testset with given sizes, and use the accuracy metric of rmse. We’ll then use the fit() method which will train the algorithm on the trainset, and the test() method which will return the predictions made from the testset

In [30]:
trainset, testset = train_test_split(data, test_size=0.25)
algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.1008


0.10077605622678701

In [27]:
predictions

[Prediction(uid='monitoring', iid='switch', r_ui=1.0, est=1.0264285065456284, details={'was_impossible': False}),
 Prediction(uid='humidity', iid='Capacitor 10 F', r_ui=1.0174757281553397, est=1.0060366584534843, details={'was_impossible': False}),
 Prediction(uid='security', iid="0.96'' OLED color display 96 * 64", r_ui=1.0, est=1.041563779322265, details={'was_impossible': False}),
 Prediction(uid='cloud arduino', iid='SparkFun Dual H-Bridge motor drivers L298', r_ui=1.0, est=1.0168396764651952, details={'was_impossible': False}),
 Prediction(uid='smart', iid='BD 135 Transistor', r_ui=1.0, est=1, details={'was_impossible': False}),
 Prediction(uid='toys', iid='Ultrasonic Sensor - HC-SR04 (Generic)', r_ui=1.192233009708738, est=1.0727338780690463, details={'was_impossible': False}),
 Prediction(uid='health', iid='Arduino MKR WAN 1300', r_ui=1.0, est=1.0173965743199251, details={'was_impossible': False}),
 Prediction(uid='analog', iid='NodeMCU ESP8266 Breakout Board', r_ui=1.0, est=1.0

In [ ]:
# dump.dump('./dump_file', predictions, algo)
# predictions, algo = dump.load('./dump_file')

In [ ]:
trainset = algo.trainset
print(algo.__class__.__name__)

To inspect our predictions in details, we are going to build a pandas data frame with all the predictions.

In [31]:
uid = str("home automation")  # raw user id (as in the ratings file). They are **strings**!
iid = str("Arduino UNO")  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, verbose=True)

user: home automation item: Arduino UNO r_ui = None   est = 1.22   {'was_impossible': False}


In [32]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)

In [33]:
df.head(100)

,uid,iid,rui,est,details,Iu,Ui,err
0,balancing,Jumper wires (generic),1.017476,1.073823,{'was_impossible': False},11,668,0.056348
1,led,Proximity sensor,1.000000,1.051744,{'was_impossible': False},393,14,0.051744
2,gesture,Modulo Joystick,1.000000,1.000000,{'was_impossible': False},32,15,0.000000
3,smartwatch,DC Power Connector Socket,1.000000,1.000322,{'was_impossible': False},25,5,0.000322
4,dice,Capacitor 1 F,1.000000,1.000000,{'was_impossible': False},28,19,0.000000
...,...,...,...,...,...,...,...,...
95,arduino,Real Time Clock (RTC),1.017476,1.046428,{'was_impossible': False},405,38,0.028952
96,scratch,Resistor 10k ohm,1.017476,1.025101,{'was_impossible': False},15,424,0.007625
97,water,Arduino MKR1000,1.034951,1.052862,{'was_impossible': False},110,157,0.017911
98,windows 10 iot core,Male/Female Jumper Wires,1.000000,1.018183,{'was_impossible': False},45,178,0.018183


In [34]:

def get_precalculated_predictions(surprise_df, column_1, column_2):
    predictions = []
    trainset, testset = train_test_split(data, test_size=0.25)
    bsl_options = {'method': 'als',
                    'n_epochs': 5,
                    'reg_u': 12,
                    'reg_i': 5
                    }
    algo = BaselineOnly(bsl_options=bsl_options)
    algo.fit(trainset).test(testset)
    display(surprise_df)
    for uid in set(surprise_df[column_1]):
        for iid in set(surprise_df[column_2]):
            #p = algo.predict(uid, iid)
            p = algo.predict("temperature", "arduino")
            predictions.append(p)
    return predictions
            
full_predictions = get_precalculated_predictions(norm_surprise_df, "tagID", "componentID")

Estimating biases using als...


,tagID,componentID,rating
110,solar tracker,Resistor 330 ohm,1.017476
162,solar tracker,Resistor 220 ohm,1.000000
248,solar tracker,Resistor 2.21k ohm,1.000000
337,solar tracker,HC-05 Bluetooth Module,1.000000
587,solar tracker,Through Hole Resistor 10 ohm,1.000000
...,...,...,...
3365832,circuit,Resistor 220 ohm,1.000000
3366017,circuit,Buzzer,1.000000
3366311,circuit,5 mm LED: Red,1.000000
3367233,circuit,Breadboard (generic),1.000000


[Prediction(uid='temperature', iid='arduino', r_ui=None, est=1.0522864383586663, details={'was_impossible': False}),
 Prediction(uid='temperature', iid='arduino', r_ui=None, est=1.0522864383586663, details={'was_impossible': False}),
 Prediction(uid='temperature', iid='arduino', r_ui=None, est=1.0522864383586663, details={'was_impossible': False}),
 Prediction(uid='temperature', iid='arduino', r_ui=None, est=1.0522864383586663, details={'was_impossible': False}),
 Prediction(uid='temperature', iid='arduino', r_ui=None, est=1.0522864383586663, details={'was_impossible': False}),
 Prediction(uid='temperature', iid='arduino', r_ui=None, est=1.0522864383586663, details={'was_impossible': False}),
 Prediction(uid='temperature', iid='arduino', r_ui=None, est=1.0522864383586663, details={'was_impossible': False}),
 Prediction(uid='temperature', iid='arduino', r_ui=None, est=1.0522864383586663, details={'was_impossible': False}),
 Prediction(uid='temperature', iid='arduino', r_ui=None, est=1.0

In [ ]:
len(full_predictions[0])

from collections import defaultdict
def get_top_n( predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.


    top_n = defaultdict(list)
    for  uid,iid, true_r, est, _ in predictions:
        top_n[uid].append((iid,true_r,est,6))

    # Then sort the predictions for each user and retrieve the k highest ones.


    display(top_n.items())
    #for iid, user_ratings, est in top_n.items():

    #    est.sort(key=lambda x: x[1], reverse=True)
    #    top_n[iid] = est[:n]

    #return top_n
top_10_recommendations = get_top_n(full_predictions, n=10)

In [39]:

display(len(top_10_recommendations))

TypeError: object of type 'NoneType' has no len()

In [ ]:
import json
def write_recommendation_to_file(top_n_recommendations,file):
    with open(file, 'w') as convert_file:
        convert_file.write(json.dumps(top_n_recommendations))

def read_recommendation_from_file(file='recommendation_top10_project_component_0_to_276_cut_5_10.json'):
    # reading the data from the file
    with open(file) as f:
        data = f.read()
    js = json.loads(data)
    return js

In [ ]:
write_recommendation_to_file(top_n_recommendations = dict(top_10_recommendations), file = 'C:\\progetti\\low-code-ard-proj\\resyduo_components\\recommender_component\\recommendation_storage\\top10_rec_surprise_tag_comp_cut_5_5_new_approach.json')

In [ ]:
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [ ]:
df.index

In [ ]:
best_predictions

The above are the best predictions, and they are not lucky guesses. Because Ui is anywhere between 26 to 146, they are not really small, meaning that significant number of users have rated the target book.

In [ ]:
worst_predictions

The worst predictions look pretty surprise. Let's look in more details of the last one ISBN "055358264X", the book was rated by 47 users, user "26544" rated 10, our BaselineOnly algorithm predicts 0.

In [ ]:
uid = str("temperature")  # raw user id (as in the ratings file). They are **strings**!
iid = str("Arduino UNO")  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid,r_ui =5.718447, verbose=True)

In [ ]:
df_new.loc[df_new['componentID'] == 'Rubber feet']['rating'].describe()

In [ ]:
df.loc[df['uid'] == 'display']

In [ ]:
df_new.groupby('tagID')['rating'].count().reset_index().sort_values('rating', ascending=True)[:100]

In [ ]:
df.loc[df['uid'] == 'pianomidi']

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook

df_new.loc[df_new['componentID'] == 'Rubber feet']['rating'].hist()
plt.xlabel('rating')
plt.ylabel('Number of ratings')
plt.title('Number of ratings item Rubber feet has received')
plt.show();

It turns out, most of the ratings this book received was "0", in another word, most of the users in the data rated this book "0", only very few users rated "10". Same with the other predictions in "worst predictions" list. It seems that for each prediction, the users are some kind of outsiders.